In [ ]:
# Instalar a última versão do PySpark
!pip install pyspark #==3.3.1

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip


# Iniciar a sessão spark
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder                  
      .config('spark.ui.port', '4050')
      .appName("SparkSQL")
      .getOrCreate()
)

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 5.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b7e27277f0c54a811d12933b59c4046278eac4121fc5d757a9cc5c010b913f38
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://5103-34-74-190-226.ngrok.io


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql.types import *

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = 'drive/MyDrive/Colab Notebooks/case_final.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd HH:mm:ss"
)

In [ ]:
df.printSchema()

df.show()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)

+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|id_transacao|             valor|           remetente|        destinatario|chave_pix|    categoria|   transaction_date|fraude|
+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|        1000|            588

In [ ]:
from pyspark.sql.functions import *

df_flatten = df.withColumns({
    'destinatario_nome': col('destinatario').getField('nome'),
    'destinatario_banco': col('destinatario').getField('banco'),
    'destinatario_tipo': col('destinatario').getField('tipo'),
}).drop('remetente', 'destinatario')

In [ ]:
df_flatten.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- destinatario_nome: string (nullable = true)
 |-- destinatario_banco: string (nullable = true)
 |-- destinatario_tipo: string (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(
    inputCols=[
        'destinatario_nome',
        'destinatario_banco',
        'destinatario_tipo',
        'categoria',
        'chave_pix'
    ],
    outputCols=[
        'destinatario_nome_index',
        'destinatario_banco_index',
        'destinatario_tipo_index',
        'categoria_index',
        'chave_pix_index'        
    ]
)

df_index = indexer.fit(df_flatten).transform(df_flatten)

In [ ]:
df_index.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- destinatario_nome: string (nullable = true)
 |-- destinatario_banco: string (nullable = true)
 |-- destinatario_tipo: string (nullable = true)
 |-- destinatario_nome_index: double (nullable = false)
 |-- destinatario_banco_index: double (nullable = false)
 |-- destinatario_tipo_index: double (nullable = false)
 |-- categoria_index: double (nullable = false)
 |-- chave_pix_index: double (nullable = false)



In [ ]:
df_index.show()

+------------+------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|   destinatario_nome|destinatario_banco|destinatario_tipo|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|categoria_index|chave_pix_index|
+------------+------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|        1000|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|         Calebe Melo|             Caixa|               PF|                12045.0|                     4.0|                    1.0|            6.0|            3.0|
|       

In [ ]:
is_fraud = df_index.filter("fraude == 1") # filter(col('fraude') == 1)
no_fraud = df_index.filter("fraude == 0")

In [ ]:
no_fraud = no_fraud.sample(False, 0.01, seed=123) 

In [ ]:
df_concat = no_fraud.union(is_fraud)
df = df_concat.sort("transaction_date")
df.count()

16202

In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed = 123)
print("train =", train.count(), " test =", test.count())

train = 11278  test = 4924


In [ ]:
is_fraud = udf(lambda fraud: 1.0 if fraud > 0 else 0.0, DoubleType())
train = train.withColumn("is_fraud", is_fraud(train.fraude))

In [ ]:
train.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- destinatario_nome: string (nullable = true)
 |-- destinatario_banco: string (nullable = true)
 |-- destinatario_tipo: string (nullable = true)
 |-- destinatario_nome_index: double (nullable = false)
 |-- destinatario_banco_index: double (nullable = false)
 |-- destinatario_tipo_index: double (nullable = false)
 |-- categoria_index: double (nullable = false)
 |-- chave_pix_index: double (nullable = false)
 |-- is_fraud: double (nullable = true)



In [ ]:
train.columns

['id_transacao',
 'valor',
 'chave_pix',
 'categoria',
 'transaction_date',
 'fraude',
 'destinatario_nome',
 'destinatario_banco',
 'destinatario_tipo',
 'destinatario_nome_index',
 'destinatario_banco_index',
 'destinatario_tipo_index',
 'categoria_index',
 'chave_pix_index',
 'is_fraud']

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

assembler = VectorAssembler(
    inputCols = [x for x in train.columns if x not in ['transaction_date', 'fraude', 'is_fraud', 'destinatario_nome', 'destinatario_banco', 'destinatario_tipo', 'chave_pix', 'categoria']],
    outputCol="features"
)

In [ ]:
lr = LogisticRegression().setParams(
    maxIter=100000,
    labelCol = "is_fraud",
    predictionCol="prediction"
)

In [ ]:
model = Pipeline(stages=[assembler, lr]).fit(train)

In [ ]:
predicted = model.transform(test)

In [ ]:
predicted.show()

+------------+--------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+--------------------+--------------------+--------------------+----------+
|id_transacao|   valor|chave_pix|    categoria|   transaction_date|fraude|   destinatario_nome|destinatario_banco|destinatario_tipo|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|categoria_index|chave_pix_index|            features|       rawPrediction|         probability|prediction|
+------------+--------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+--------------------+--------------------+--------------------+----------+
|        1011|21345.91|      cpf|transferencia|2021-10-31 04:31:

In [ ]:
predicted = predicted.withColumn('is_fraud', is_fraud(predicted.fraude))
predicted.crosstab('is_fraud', 'prediction').show()

+-------------------+---+----+
|is_fraud_prediction|0.0| 1.0|
+-------------------+---+----+
|                1.0|  0|4682|
|                0.0|242|   0|
+-------------------+---+----+



In [ ]:
df_teste_cols = [
    'id_transacao',
    'valor', 
    'transaction_date',
    'destinatario_nome_index',
    'destinatario_banco_index',
    'destinatario_tipo_index',
    'chave_pix_index',
    'categoria_index',
    'fraude'
]

df_teste_data = [
    (999,103.2, "2023-01-01 11:56:41", 328.0, 4.0, 1.0, 3.0, 5.0, 0),
    (998, 500000.0, "2023-01-01 11:56:41", 328.0, 2.0, 3.0, 2.0, 5.0, 1),
    (997, 19999.0, "2023-01-01 11:56:41", 328.0, 1.0, 2.0, 1.0, 5.0, 0),
]

df_teste = spark.createDataFrame(df_teste_data).toDF(*df_teste_cols)

In [ ]:
new_prediction = model.transform(df_teste)

In [ ]:
new_prediction.show()

+------------+--------+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+--------------------+--------------------+-----------+----------+
|id_transacao|   valor|   transaction_date|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|chave_pix_index|categoria_index|fraude|            features|       rawPrediction|probability|prediction|
+------------+--------+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+--------------------+--------------------+-----------+----------+
|         999|   103.2|2023-01-01 11:56:41|                  328.0|                     4.0|                    1.0|            3.0|            5.0|     0|[999.0,103.2,328....|[426.491663444370...|  [1.0,0.0]|       0.0|
|         998|500000.0|2023-01-01 11:56:41|                  328.0|                     2.0|                    3.0|